<a href="https://colab.research.google.com/github/AdityaArakere/Sentiment-analysis-model/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import string
import re
from google.colab import files


uploaded = files.upload()


Saving combined_dataset6.csv to combined_dataset6.csv


In [ ]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['combined_dataset6.csv']))
df = df[['text', 'sentiment']]

# Preprocess the dataset
# Convert sentiment labels to 0, 1, -1
# df['category'] = df['category'].map({-1: [1, 2], 0: [3, 0], 1: [4, 5]})

# Convert text to lowercase
df['text'] = df['text'].str.lower()

# Remove punctuation
translator = str.maketrans('', '', string.punctuation)
df['text'] = df['text'].astype(str).apply(lambda x: x.translate(translator))

# Remove numbers
df['text'] = df['text'].apply(lambda x: re.sub(r'\d+', '', x))

# Remove extra whitespaces
df['text'] = df['text'].apply(lambda x: ' '.join(x.split()))

# Remove stopwords (optional)
# You can use NLTK or other libraries to remove stopwords if desired

# Print the preprocessed dataset
print(df.head())


                                                text  sentiment
0  one of the other reviewers has mentioned that ...          1
1  a wonderful little production br br the filmin...          1
2  i thought this was a wonderful way to spend ti...          1
3  petter matteis love in the time of money is a ...          1
4  probably my alltime favorite movie a story of ...          1


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
!pip install transformers
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from transformers import get_linear_schedule_with_warmup

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 107.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.9 MB/s eta 0:00:00


In [ ]:
# Split the dataset into train and test sets
x = df['text'].values
y = df['sentiment'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=2000, test_size=400, random_state=42)

# Print the sizes of the train and test sets
print("Train set size:", len(x_train))
print("Test set size:", len(x_test))
print(y_train[0:5])

Train set size: 2000
Test set size: 400
[0 0 0 1 1]


In [ ]:
# Define a custom dataset class
class RedditDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoded_inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )

        input_ids = encoded_inputs['input_ids'].squeeze()
        attention_mask = encoded_inputs['attention_mask'].squeeze()

        return input_ids, attention_mask, label

# Create an instance of the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Create train and test datasets with modified labels
train_dataset = RedditDataset(x_train, y_train, tokenizer)
test_dataset = RedditDataset(x_test, y_test, tokenizer)

# Create data loaders
train_data_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=16)

# Load the pre-trained BERT model for sentiment analysis
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.dropout = nn.Dropout(0.5)  # Add a dropout layer with dropout rate of 0.2

# Define loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)  # Add weight decay of 0.01

# Define the number of training steps
num_epochs = 3
total_steps = len(train_data_loader) * num_epochs

# Create the learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

model.train()
for epoch in range(num_epochs):
    total_loss = 0

    for input_ids, attention_mask, labels in train_data_loader:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        # print(input_ids)
        # print(labels)
        # labels = torch.cat([t.to(device) for t in labels])

        optimizer.zero_grad()

        # batch_size = input_ids.size(0)
        # if labels.size(0) != batch_size:
        #     labels = labels[:batch_size]

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()


    avg_loss = total_loss / len(train_data_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')

# Evaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for input_ids, attention_mask, labels in test_data_loader:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        # labels = torch.cat([t.to(device) for t in labels])

        # batch_size = input_ids.size(0)
        # if labels.size(0) != batch_size:
        #   labels = labels[:batch_size]

        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, dim=1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Epoch 1/3, Loss: 0.6061
Epoch 2/3, Loss: 0.3467
Epoch 3/3, Loss: 0.1247
Test Accuracy: 79.00%


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the paths to save the model and tokenizer
model_path = '/content/drive/MyDrive/Models/fine_tuned_model9'
tokenizer_path = '/content/drive/MyDrive/Models/tokenizer9'

# Save the fine-tuned model
model.save_pretrained(model_path)

# Save the tokenizer
tokenizer.save_pretrained(tokenizer_path)


('/content/drive/MyDrive/Models/tokenizer9/tokenizer_config.json',
 '/content/drive/MyDrive/Models/tokenizer9/special_tokens_map.json',
 '/content/drive/MyDrive/Models/tokenizer9/vocab.txt',
 '/content/drive/MyDrive/Models/tokenizer9/added_tokens.json')

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load the saved fine-tuned model and tokenizer
model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/Models/fine_tuned_model9')
tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/Models/tokenizer9')

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Prepare the input text
sentence = "can you help activating the product"

# Tokenize the input text
tokens = tokenizer.encode(sentence, return_tensors="pt")
result = model(tokens)
sentiment_label = int(torch.argmax(result.logits))

if sentiment_label == 0:
      final_sent = "negative"
else:
      final_sent = "positive"

# if sentiment_label < 3:
#     final_sent="negative"
# elif sentiment_label >3:
#     final_sent="positive"
# else:
#     final_sent="neutral"

# print(final_sent)
print(final_sent)

positive
